### 观察者模式

有时候，我们想要在另外一个对象改变时更新一组对象。一个非常流行的例子就是模型－视图－控制器（MVC）模式。假设，我们在两个视图中使用了相同模型的数据，比如，一个表格中的饼状图。不论模型是否被修改，这两个视图都需要更新。这正是观察者设计模式所扮演的角色。
观察者模式描述了单一对象之间的发布－订阅关系，发布者也称作主题或者可观察对象，并且拥有一个或者更多的对象，订阅者，也称作观察者。在MVC例子中，发布者为模型，订阅者为视图。不过，MVC并不是唯一的发布－订阅例子。订阅新闻聚合，比如RSS或者Atom就是另外一个例子了。很多读者通常使用资源阅读器订阅资源，每次添加新资源时，他们都可以自动地收到更新。
观察者背后的思想是和MVC背后的思想以及关注原则的分离是相同的，即，加强发布者和订阅者之间的耦合拆解，并在运行时可以很容易地添加／删除订阅者。此外，发布者并不关心自己的观察者是谁。它只会把通知发送给全部的订阅者。

在现实中，拍卖就很像观察者。每个拍卖投标者都拥一定数量的不论何时他们想要出价时要举起的报价牌。不论什么时候，报价牌被投标者举起时，拍卖主持人在更新投标价格时扮演者被监视对象的角色，同时会把新报价广播给全部的投标者（订阅者）。

使用案例
我们一般在想要通知／更新一个或者更多对象有变更发生在了另外一个对象（主题／发布者／可订阅对象）身上。观察者的数量也随着观察者在改变，而且可以动态地改变（在运行时）。

我可以思考一下，在很多场合观察者都是非常有用的。其中一个此类场景在本章开始就已经提到了：新闻订阅。不论是RSS，Atom，或者其它格式，其思想都是一样的；你关注了订阅，每次订阅更新时，你就可以收到更新通知。

同样的概念也存在社交网络中。如果你使用社交网络服务连接到了另外一个人，只要你的连接对象有内容更新，那么你也会得到通知。如果的连接的人是推特用户，脸书上的真实朋友，或者LinkedIn上的业务伙伴。

时间驱动的系统是另外一种情况，这里观察者也可（通常）被使用。在此类系统中，侦听者用来“侦听”特定的事件。这可以是特定（键盘）按键的键入，移动鼠标，等等。事件扮演了发布者的角色，侦听者扮演了观察者的角色。这种情况的关键点在于存在多个侦听者（以及观察者）加入到了单一的事件（发布者）中。

### 实现      
在本节，我们要实现一个数据格式器。
观察者实际上是通过继承实现的模式其中一种。我们可以基本的Publisher类，它包含了常用的添加，移除，以及通知观察者的功能。我们的DefaultFormatter类派生自Publisher，而且添加了格式器专用的功能。我们可以按照需求动态地添加和移除观察者。下面的类图表展示了使用两个观察者例子的实例：HexFormatter 和BinaryFormatter。注意，因为类图表是静态的，所以它们不能够显示整个系统的生命周期内的东西，只展示了在特定时间内的部分状态。
我们从Publisher类开始。观察者保存在了observers列表中。add()方法注册了一个新的观察者，或者在观察者已经存在的情况下抛出一个错误。 remove()注销一个现有的观察者，或者在没有观察者存在时抛出一个异常。最后，notify()方法对所有的观察者通知变更的发生：

In [1]:
class Publisher:

    def __init__(self):
        self.observers = []

    def add(self, observer):
        if observer not in self.observers:
            self.observers.append(observer)
        else:
            print('Failed to add: {}'.format(observer))

    def remove(self, observer):
        try:
            self.observers.remove(observer)
        except ValueError:
            print('Failed to remove: {}'.format(observer))

    def notify(self):
        [o.notify(self) for o in self.observers]

我们继续编写DefaultFormatter类。 __init__()做的第一件事就是调用基类的 __init__()方法，因为在Python中这个操作并不会自动完成。DefaultFormatter实力拥有name，以便让我们更容易地跟踪它的状态。我们在_data变量中使用名称扭曲来声明这个变量不应该被直接访问。

In [4]:
class DefaultFormatter(Publisher):

    def __init__(self, name):
        Publisher.__init__(self)
        self.name = name
        self._data = 0
        

    def __str__(self):
        """
        __str__()方法返回发布者的名称信息， _data. type(self).__name__的值是一个获取
        类名称而不用硬编码的技巧。这是让代码可读性下降，但是易于维护的做法中其中一个。
        这样做，喜不喜欢完全由你自己决定
        """
        return "{}: '{}' has data = {}".format(type(self).__name__, self.name, self._data)

    @property
    def data(self):
        """
        有两个data()方法。第一个方法使用@property装饰器给出了对 _data变量的读访问。使用
        该装饰器，我们只要执行object.data而不是object.data()就行了
        """
        return self._data

    @data.setter
    def data(self, new_value):
        """
        第二个data()方法就更有意思了。它使用了 @setter 装饰器，该装饰器在每次赋值运算符（＝）
        用来将一个新值赋予_data变量时被调用。该方法也试着把一个新值转换为整数，
        并在操作失败的情况下执行异常处理
        """
        try:
            self._data = int(new_value)
        except ValueError as e:
            print('Error: {}'.format(e))
        else:
            self.notify()

下一步是添加观察者。HexFormatter和BinaryFormatter的功能非常相似。唯一区别在于，它们之间如何格式化发布者接收到值，即，分别使用了十进制和二进制

In [5]:
class HexFormatter:

    def notify(self, publisher):
        print("{}: '{}' has now hex data = {}".format(type(self).__name__,
                                                      publisher.name, hex(publisher.data)))


class BinaryFormatter:

    def notify(self, publisher):
        print("{}: '{}' has now bin data = {}".format(type(self).__name__,
                                                      publisher.name, bin(publisher.data)))


例子在离开测试数据后会显得毫无乐趣。main()函数初始化时创建了DefaultFormatter的命名为test1实例，之后附加（和移除）了两个可用的观察者。异常处理也被使用了，以保证用户在传输错误数据时不会让应用崩溃。此外，添加相同的观察者两次，或者移除一个不存在的观察者这类尝试并不会引起崩溃

In [8]:
class Publisher:

    def __init__(self):
        self.observers = []

    def add(self, observer):
        if observer not in self.observers:
            self.observers.append(observer)
        else:
            print('Failed to add: {}'.format(observer))

    def remove(self, observer):
        try:
            self.observers.remove(observer)
        except ValueError:
            print('Failed to remove: {}'.format(observer))

    def notify(self):
        [o.notify(self) for o in self.observers]


class DefaultFormatter(Publisher):

    def __init__(self, name):
        Publisher.__init__(self)
        self.name = name
        self._data = 0

    def __str__(self):
        return "{}: '{}' has data = {}".format(type(self).__name__, 
                                               self.name, self._data)

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, new_value):
        try:
            self._data = int(new_value)
        except ValueError as e:
            print('Error: {}'.format(e))
        else:
            self.notify()


class HexFormatter:

    def notify(self, publisher):
        print("{}: '{}' has now hex data = {}".format(type(self).__name__,
                                                      publisher.name, hex(publisher.data)))


class BinaryFormatter:

    def notify(self, publisher):
        print("{}: '{}' has now bin data = {}".format(type(self).__name__,
                                                      publisher.name, bin(publisher.data)))


def main():
    df = DefaultFormatter('test1')
    print(df)

    print()
    hf = HexFormatter()
    df.add(hf)
    df.data = 3
    print(df)

    print()
    bf = BinaryFormatter()
    df.add(bf)
    df.data = 21
    print(df)

    print()
    df.remove(hf)
    df.data = 40
    print(df)

    print()
    df.remove(hf)
    df.add(bf)
    df.data = 'hello'
    print(df)

    print()
    df.data = 15.8
    print(df)

In [9]:
main()

DefaultFormatter: 'test1' has data = 0

HexFormatter: 'test1' has now hex data = 0x3
DefaultFormatter: 'test1' has data = 3

HexFormatter: 'test1' has now hex data = 0x15
BinaryFormatter: 'test1' has now bin data = 0b10101
DefaultFormatter: 'test1' has data = 21

BinaryFormatter: 'test1' has now bin data = 0b101000
DefaultFormatter: 'test1' has data = 40

Failed to remove: <__main__.HexFormatter object at 0x0000000004E953C8>
Failed to add: <__main__.BinaryFormatter object at 0x0000000004E95400>
Error: invalid literal for int() with base 10: 'hello'
DefaultFormatter: 'test1' has data = 40

BinaryFormatter: 'test1' has now bin data = 0b1111
DefaultFormatter: 'test1' has data = 15


django-observer包是一个第三方Django包，它用作在Django字段发生多处改变时注册被执行的回调函数。这个包支持很多不同类型的字段（CharField, IntegerField，等等）。

RabbitMQ 是一个可以用来对一个应用添加异步消息支持的库。它支持多种消息协议，比如HTTP和AMQP。RabbitMQ用在Python应用中可以实现发布－订阅模式，比起观察者设计模式它什么也是不。